# Feature Engineering

# References

* http://www.sciencedirect.com/science/article/pii/S0925231203003722
* http://ac.els-cdn.com/S0957417400000270/1-s2.0-S0957417400000270-main.pdf?_tid=3a06fc62-1d5b-11e6-877f-00000aab0f27&acdnat=1463619013_cba9f7ee840313639128ce15571f73ac
* Technical Analysis of Stock Trends, Robert D. Edwards and John Magee
* https://www.jbs.cam.ac.uk/fileadmin/user_upload/research/workingpapers/wp0030.pdf
* http://www.sciencedirect.com/science/article/pii/0261560692900483
* https://www.quantopian.com/posts/technical-analysis-indicators-without-talib-code

In [3]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information numpy, pandas

Software versions
Python 3.5.1 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.1.2
OS Linux 3.13.0 86 generic x86_64 with debian jessie sid
numpy 1.10.4
pandas 0.18.0
Wed May 25 12:19:24 2016 BST

In [4]:
%matplotlib inline

import ast
import numpy as np
import pandas as pd

np.set_printoptions(threshold=np.nan)

# Technical Indicator Functions

In [5]:
def stoch_K(close, window):
    low = close.rolling(window, center = False).min()
    high = close.rolling(window, center = False).max()
    
    return 100 * (close - low) / (high - low)

def stoch_D(K, window):
    return K.rolling(window, center = False).mean()

def slow_D(D, window):
    return D.rolling(window, center = False).mean()

def momentum(close, window):
    dif = lambda x: x[-1] - x[0]
    
    return close.rolling(window, center = False).apply(dif)

def roc(close, window):
    ratio = lambda x: x[-1] / x[0]
    
    return 100 * close.rolling(window, center = False).apply(ratio)

def lw_R(close, window):
    low = close.rolling(window, center = False).min()
    high = close.rolling(window, center = False).max()
    
    return 100 * (high - close) / (high - low)
    
def ad_osc(close, window):
    low = close.rolling(window, center = False).min()
    high = close.rolling(window, center = False).max()
    prev_close = close.rolling(window, center = False).apply(lambda x: x[-2])
    
    return (high - prev_close) / (high - low)

def disp(close, window):
    MA = close.rolling(window, center = False).mean()
    
    return 100 * close / MA

def oscp(close, window1, window2):
    MA1 = close.rolling(window1, center = False).mean()
    MA2 = close.rolling(window2, center = False).mean()
    
    return (MA1 - MA2) / MA1

def rsi(close, window):
    up, down = close.diff().copy(), close.diff().copy()
    up[up < 0] = 0
    down[down > 0] = 0
    
    RS = up.rolling(window, center = False).mean() / down.rolling(window, center = False).mean().abs()
    
    return 100 - (100 / (1 + RS))

def cci(close, window):
    low = close.rolling(window, center = False).min()
    high = close.rolling(window, center = False).max()
    
    MT = (close + low + high) / 3
    SMT = MT.rolling(window, center = False).mean()
    DT = MT.rolling(window, center = False).std()
    
    return (MT - SMT) / DT

# Feature Engineering of LOB Feature Set 1 - simple price and volume features

In [158]:
lob_data = pd.read_csv('../btc-data/BTC_LOB_collected.csv')

In [159]:
lob_features10 = pd.DataFrame(lob_data)
lob_features10.set_index(lob_data['Unnamed: 0'], inplace = True)
lob_features10.drop('Unnamed: 0', axis = 1, inplace = True)
lob_features10.index = pd.to_datetime(lob_features10.index)
lob_features10['asks'] = lob_features10['asks'].map(ast.literal_eval)
lob_features10['bids'] = lob_features10['bids'].map(ast.literal_eval)

In [160]:
lob_features10['total ask volume'] = lob_features10['asks'].map(lambda x: sum(x.values()))
lob_features10['total bid volume'] = lob_features10['bids'].map(lambda x: sum(x.values()))
lob_features10['ask price'] = lob_features10['asks'].map(min)
lob_features10['bid price'] = lob_features10['bids'].map(max)

In [161]:
lob_features10['bid-ask spread'] = lob_features10['ask price'] - lob_features10['bid price']
lob_features10['mid price'] = (lob_features10['ask price'] + lob_features10['bid price'])/2
lob_features10['ask price spread'] = lob_features10['asks'].map(max) - lob_features10['ask price']
lob_features10['bid price spread'] = lob_features10['bid price'] - lob_features10['bids'].map(min)
lob_features10['mean ask volume'] = lob_features10['total ask volume'] / 20
lob_features10['mean bid volume'] = lob_features10['total bid volume'] / 20
lob_features10['mean ask price'] = lob_features10['asks'].map(sum) / 20
lob_features10['mean bid price'] = lob_features10['bids'].map(sum) / 20

In [162]:
lob_features10

,asks,bids,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
Unnamed: 0,,,,,,,,,,,,,,
2016-01-01 00:00:00,"{432.84: 39.75, 432.36: 6.9631, 432.83: 9.2466...","{429.31: 3.88, 429.33: 39.75, 429.47: 32.0333,...",318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:00:10,"{432.88: 41.73, 432.48: 13.8492, 432.99: 9.257...","{429.38: 16.114, 429.2: 48.9702, 429.33: 39.75...",318.629099,323.428038,430.89,429.84,1.05,430.365,2.11,1.19,15.931455,16.171402,432.1875,429.1115
2016-01-01 00:00:20,"{432.64: 6.9491, 432.52: 1.17, 432.84: 39.75, ...","{429.41: 9.2521, 429.33: 39.75, 429.01: 14.836...",290.919269,354.135003,430.89,429.86,1.03,430.375,2.44,1.54,14.545963,17.706750,432.2465,429.0100
2016-01-01 00:00:30,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.33: 39.75, 429.35: 16...",232.153855,172.031328,430.89,429.89,1.00,430.390,2.80,1.56,11.607693,8.601566,432.3350,429.0750
2016-01-01 00:00:40,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{429.49: 32.0333, 428.65: 0.02799486, 428.83: ...",290.952369,322.855038,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,16.142752,432.2360,429.1115
2016-01-01 00:00:50,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.45: 39.75, 429.28: 0....",290.952369,287.730638,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,14.386532,432.2360,429.1010
2016-01-01 00:01:00,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.2: 48.9764, 429.01: 1...",290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:01:10,"{432.11: 4.635, 432.35: 6.9594000000000005, 43...","{429.45: 39.75, 429.47: 14.916, 428.65: 0.0279...",290.875538,322.244428,430.89,429.90,0.99,430.395,2.44,1.55,14.543777,16.112221,432.2355,429.0825
2016-01-01 00:01:20,"{432.11: 4.635, 432.35: 6.8916, 433.0: 0.1, 43...","{429.02: 29.13, 429.91: 32.0333, 429.47: 2.35,...",279.887738,309.666428,430.89,429.91,0.98,430.400,2.32,1.56,13.994387,15.483321,432.2295,429.1055


In [163]:
lob_features10.drop(['asks', 'bids'], axis = 1, inplace=True)
lob_features10.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_10s.csv')

In [164]:
lob_features30 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                      end = lob_features10.index[-1], freq='30s'))

lob_features60 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                      end = lob_features10.index[-1], freq='60s'))

lob_features300 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                       end = lob_features10.index[-1], freq='300s'))

lob_features600 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                       end = lob_features10.index[-1], freq='600s'))

In [171]:
lob_features30

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:00:30,232.153855,172.031328,430.89,429.89,1.00,430.390,2.80,1.56,11.607693,8.601566,432.3350,429.0750
2016-01-01 00:01:00,290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:01:30,285.901138,307.380028,430.89,429.92,0.97,430.405,2.44,1.57,14.295057,15.369001,432.3270,429.0700
2016-01-01 00:02:00,312.929338,327.741478,430.89,429.95,0.94,430.420,2.28,1.30,15.646467,16.387074,432.3600,429.1935
2016-01-01 00:02:30,312.832581,326.718378,430.89,430.05,0.84,430.470,2.28,1.40,15.641629,16.335919,432.3450,429.2110
2016-01-01 00:03:00,279.716681,336.122423,430.89,430.09,0.80,430.490,2.28,1.41,13.985834,16.806121,432.3365,429.2315
2016-01-01 00:03:30,279.744681,306.098773,430.89,430.14,0.75,430.515,2.28,1.44,13.987234,15.304939,432.3665,429.3355
2016-01-01 00:04:00,279.871681,295.116008,430.89,430.17,0.72,430.530,2.33,1.52,13.993584,14.755800,432.3890,429.2450
2016-01-01 00:04:30,231.934879,179.662946,430.89,430.19,0.70,430.540,2.61,2.07,11.596744,8.983147,432.4435,428.9195


In [172]:
lob_features60

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:01:00,290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:02:00,312.929338,327.741478,430.89,429.95,0.94,430.420,2.28,1.30,15.646467,16.387074,432.3600,429.1935
2016-01-01 00:03:00,279.716681,336.122423,430.89,430.09,0.80,430.490,2.28,1.41,13.985834,16.806121,432.3365,429.2315
2016-01-01 00:04:00,279.871681,295.116008,430.89,430.17,0.72,430.530,2.33,1.52,13.993584,14.755800,432.3890,429.2450
2016-01-01 00:05:00,217.439947,342.194493,431.06,430.19,0.87,430.625,2.74,1.50,10.871997,17.109725,432.6220,429.2145
2016-01-01 00:06:00,361.630784,294.113648,432.40,430.54,1.86,431.470,1.56,1.71,18.081539,14.705682,433.3085,429.6675
2016-01-01 00:07:00,347.402014,395.102693,432.52,431.46,1.06,431.990,1.45,2.11,17.370101,19.755135,433.3250,430.2590
2016-01-01 00:08:00,347.029411,338.648408,432.53,431.46,1.07,431.995,1.44,2.10,17.351471,16.932420,433.3880,430.4625
2016-01-01 00:09:00,414.224478,267.615308,432.53,431.46,1.07,431.995,1.43,2.45,20.711224,13.380765,433.3280,430.1980


In [173]:
lob_features300

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:05:00,217.439947,342.194493,431.06,430.19,0.87,430.625,2.74,1.50,10.871997,17.109725,432.6220,429.2145
2016-01-01 00:10:00,414.543886,305.888470,432.41,431.46,0.95,431.935,1.49,2.53,20.727194,15.294424,433.1320,430.1110
2016-01-01 00:15:00,183.180919,222.309648,432.38,431.46,0.92,431.920,1.49,2.67,9.159046,11.115482,433.0830,430.0345
2016-01-01 00:20:00,311.297511,398.569889,432.42,431.46,0.96,431.940,1.47,2.45,15.564876,19.928494,433.2915,430.2975
2016-01-01 00:25:00,419.611941,434.477720,432.40,431.46,0.94,431.930,1.52,2.44,20.980597,21.723886,433.2520,430.1780
2016-01-01 00:30:00,234.429204,264.975279,432.39,431.46,0.93,431.925,1.41,2.45,11.721460,13.248764,433.0735,430.3140
2016-01-01 00:35:00,427.122203,306.567417,432.40,430.45,1.95,431.425,1.50,1.62,21.356110,15.328371,433.1205,429.7195
2016-01-01 00:40:00,248.873168,253.628860,431.11,430.09,1.02,430.600,2.35,1.41,12.443658,12.681443,432.6055,429.3755
2016-01-01 00:45:00,216.691350,225.566000,431.02,430.46,0.56,430.740,2.77,1.67,10.834567,11.278300,432.7370,429.4195


In [174]:
lob_features600

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:10:00,414.543886,305.888470,432.41,431.46,0.95,431.935,1.49,2.53,20.727194,15.294424,433.1320,430.1110
2016-01-01 00:20:00,311.297511,398.569889,432.42,431.46,0.96,431.940,1.47,2.45,15.564876,19.928494,433.2915,430.2975
2016-01-01 00:30:00,234.429204,264.975279,432.39,431.46,0.93,431.925,1.41,2.45,11.721460,13.248764,433.0735,430.3140
2016-01-01 00:40:00,248.873168,253.628860,431.11,430.09,1.02,430.600,2.35,1.41,12.443658,12.681443,432.6055,429.3755
2016-01-01 00:50:00,254.093468,239.650073,431.05,430.16,0.89,430.605,2.64,1.81,12.704673,11.982504,432.7790,429.2280
2016-01-01 01:00:00,280.695908,189.337964,431.05,429.80,1.25,430.425,2.12,1.46,14.034795,9.466898,432.2610,429.0650
2016-01-01 01:10:00,254.614361,360.892833,431.35,429.43,1.92,430.390,2.45,1.11,12.730718,18.044642,432.5795,428.8460
2016-01-01 01:20:00,248.382690,304.781962,431.38,430.82,0.56,431.100,2.31,2.14,12.419135,15.239098,432.5520,429.2240
2016-01-01 01:30:00,257.805188,214.429454,431.23,429.80,1.43,430.515,2.57,1.50,12.890259,10.721473,432.4005,428.8950


In [175]:
lob_features30.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_30s.csv')
lob_features60.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_60s.csv')
lob_features300.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_300s.csv')
lob_features600.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_600s.csv')

# Feature Engineering of LOB Feature Set 2 - better technical indicators

In [ ]:
lob_data = pd.read_csv('../btc-data/BTC_LOB_collected.csv')

In [274]:
lob_techind10 = pd.DataFrame(lob_features10['mid price'].copy(), index = lob_features10.index)
lob_techind10['B-ASPREAD'] = lob_features10['bid-ask spread'].copy()

lob_techind30 = pd.DataFrame(lob_features30['mid price'].copy(), index = lob_features30.index)
lob_techind30['B-ASPREAD'] = lob_features30['bid-ask spread'].copy()

lob_techind60 = pd.DataFrame(lob_features60['mid price'].copy(), index = lob_features60.index)
lob_techind60['B-ASPREAD'] = lob_features60['bid-ask spread'].copy()

lob_techind300 = pd.DataFrame(lob_features300['mid price'].copy(), index = lob_features300.index)
lob_techind300['B-ASPREAD'] = lob_features300['bid-ask spread'].copy()

lob_techind600 = pd.DataFrame(lob_features600['mid price'].copy(), index = lob_features600.index)
lob_techind600['B-ASPREAD'] = lob_features600['bid-ask spread'].copy()

In [275]:
def generate_features(frame, freq):
    close = frame['mid price']
    frame['K360'] = stoch_K(close, 360)
    frame['K180'] = stoch_K(close, 180)
    frame['K60'] = stoch_K(close, 60)
    frame['D360'] = stoch_D(frame['K360'], 360)
    frame['D180'] = stoch_D(frame['K180'], 180)
    frame['D60'] = stoch_D(frame['K60'], 60)
    frame['sD360'] = slow_D(frame['D360'], 360)
    frame['sD180'] = slow_D(frame['D180'], 180)
    frame['sD60'] = slow_D(frame['D60'], 60)
    frame['MOM360'] = momentum(close, 360)
    frame['MOM180'] = momentum(close, 180)
    frame['MOM60'] = momentum(close, 60)
    frame['ROC360'] = roc(close, 360)
    frame['ROC180'] = roc(close, 180)
    frame['ROC60'] = roc(close, 60)
    frame['LWR360'] = lw_R(close, 360)
    frame['LWR180'] = lw_R(close, 180)
    frame['LWR60'] = lw_R(close, 60)
    frame['ADOSC360'] = ad_osc(close, 360)
    frame['ADOSC180'] = ad_osc(close, 180)
    frame['ADOSC60'] = ad_osc(close, 60)
    frame['DISP360'] = disp(close, 360)
    frame['DISP180'] = disp(close, 180)
    frame['DISP60'] = disp(close, 60)
    frame['OSCP180-360'] = oscp(close, 180, 360)
    frame['OSCP60-180'] = oscp(close, 60, 180)
    frame['RSI360'] = rsi(close, 360)
    frame['RSI180'] = rsi(close, 180)
    frame['RSI60'] = rsi(close, 60)
    frame['CCI360'] = cci(close, 360)
    frame['CCI180'] = cci(close, 180)
    frame['CCI60'] = cci(close, 60)
    frame['DELTAP'] = close.diff()
    
    frame['mid price'] = frame['mid price'].shift(-1)
    frame['B-ASPREAD'] = frame['B-ASPREAD'].shift(-1)
    frame['DELTAP'] = frame['DELTAP'].shift(-1)
    frame.set_index(frame.index.shift(1, freq=freq), inplace = True)
    frame = frame[3*359:-1]
    
    return frame

In [276]:
lob_techind10 = generate_features(lob_techind10, '10s')
lob_techind10.replace([np.inf, -np.inf], np.nan, inplace = True)
lob_techind10.fillna(method='ffill', inplace = True)

In [277]:
lob_techind30 = generate_features(lob_techind30, '30s')
lob_techind30.replace([np.inf, -np.inf], np.nan, inplace = True)
lob_techind30.fillna(method='ffill', inplace = True)

In [278]:
lob_techind60 = generate_features(lob_techind60, '60s')
lob_techind60.replace([np.inf, -np.inf], np.nan, inplace = True)
lob_techind60.fillna(method='ffill', inplace = True)

In [279]:
lob_techind300 = generate_features(lob_techind300, '300s')
lob_techind300.replace([np.inf, -np.inf], np.nan, inplace = True)
lob_techind300.fillna(method='ffill', inplace = True)

In [280]:
lob_techind600 = generate_features(lob_techind600, '600s')
lob_techind600.replace([np.inf, -np.inf], np.nan, inplace = True)
lob_techind600.fillna(method='ffill', inplace = True)

In [281]:
lob_techind10.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_10s.csv')
lob_techind30.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_30s.csv')
lob_techind60.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_60s.csv')
lob_techind300.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_300s.csv')
lob_techind600.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_600s.csv')

In [5]:
path = '../btc-data/BTC_LOB_techind_10s.csv'
data10s = pd.read_csv(path, index_col = 0, parse_dates = True)

path = '../btc-data/BTC_LOB_techind_30s.csv'
data30s = pd.read_csv(path, index_col = 0, parse_dates = True)

path = '../btc-data/BTC_LOB_techind_60s.csv'
data1m = pd.read_csv(path, index_col = 0, parse_dates = True)

path = '../btc-data/BTC_LOB_techind_300s.csv'
data5m = pd.read_csv(path, index_col = 0, parse_dates = True)

path = '../btc-data/BTC_LOB_techind_600s.csv'
data10m = pd.read_csv(path, index_col = 0, parse_dates = True)

In [6]:
data10s

,mid price,B-ASPREAD,K360,K180,K60,D360,D180,D60,sD360,sD180,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-01-01 02:59:40,430.305,0.53,0.000000,0.000000,0.000000,61.295468,34.942988,18.297894,45.643462,55.378159,...,99.943676,0.000199,-0.000725,49.069940,47.401247,41.388401,0.654694,-1.475509,-0.531521,0.000
2016-01-01 02:59:50,430.305,0.53,0.000000,0.000000,0.000000,61.178695,34.452537,17.479712,45.747607,55.188824,...,99.947062,0.000193,-0.000751,49.069940,46.568110,42.166517,0.653150,-1.471843,-0.506070,0.000
2016-01-01 03:00:00,430.305,0.53,0.000000,0.000000,0.000000,61.061923,33.975106,16.510015,45.851132,54.996227,...,99.950544,0.000186,-0.000778,49.069940,46.641988,41.906475,0.651610,-1.468691,-0.480539,0.000
2016-01-01 03:00:10,430.305,0.53,0.000000,0.000000,0.000000,60.945150,33.510697,15.419106,45.954035,54.800515,...,99.954104,0.000180,-0.000805,49.069940,46.716102,41.696751,0.650072,-1.466055,-0.454877,0.000
2016-01-01 03:00:20,430.305,0.53,0.000000,0.000000,0.000000,60.827787,33.146113,14.206984,46.056313,54.602805,...,99.957742,0.000175,-0.000835,49.050986,47.292225,41.485507,0.648566,-1.467540,-0.429076,0.000
2016-01-01 03:00:30,430.305,0.53,0.000000,0.000000,0.000000,60.711014,32.850975,12.903954,46.158406,54.403860,...,99.961438,0.000171,-0.000867,49.069248,47.701460,41.326067,0.647034,-1.472187,-0.403154,0.000
2016-01-01 03:00:40,430.310,0.54,0.000000,0.000000,0.000000,60.613704,32.538475,11.419106,46.260535,54.203146,...,99.965250,0.000166,-0.000899,49.679608,47.588076,41.004566,0.644613,-1.476047,-0.377549,0.005
2016-01-01 03:00:50,430.305,0.53,0.369004,0.369004,0.414938,60.494641,32.219344,9.850264,46.362579,54.000919,...,99.970263,0.000162,-0.000932,49.065242,47.559653,40.896615,0.652030,-1.469277,-0.338602,-0.005
2016-01-01 03:01:00,430.305,0.53,0.000000,0.000000,0.000000,60.374554,31.893823,8.183597,46.464510,53.797119,...,99.973050,0.000157,-0.000965,49.046529,47.505423,40.642202,0.640786,-1.483000,-0.332817,0.000
2016-01-01 03:01:10,430.305,0.53,0.000000,0.000000,0.000000,60.220923,31.559622,8.183597,46.566142,53.591649,...,99.974366,0.000153,-0.000973,49.046529,47.448426,46.436059,0.636933,-1.485904,-0.308881,0.000


In [7]:
data30s

,mid price,B-ASPREAD,K360,K180,K60,D360,D180,D60,sD360,sD180,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-01-01 08:59:00,433.805,0.75,55.862726,48.268156,50.000000,59.314645,53.892303,80.335556,73.952213,66.199069,...,99.988818,0.000376,-0.000461,48.543689,44.399799,54.328358,0.058659,-0.188518,1.015588,0.125
2016-01-01 08:59:30,433.355,1.47,58.245949,51.061453,70.161290,59.236089,53.620422,79.924971,73.885263,66.188861,...,100.016908,0.000356,-0.000427,48.692265,44.746693,52.923077,0.143537,-0.040078,1.160289,-0.450
2016-01-01 09:00:00,433.655,1.05,49.666349,41.005587,0.000000,59.136247,53.317976,78.431759,73.817585,66.176271,...,99.913713,0.000337,-0.000403,48.262548,44.271605,47.977685,-0.157779,-0.541918,0.483324,0.300
2016-01-01 09:00:30,433.675,0.91,55.386082,47.709497,47.244094,59.052293,53.041470,77.578889,73.749221,66.161379,...,99.983035,0.000318,-0.000377,48.602740,44.803130,49.457995,0.045071,-0.194077,0.900046,0.020
2016-01-01 09:01:00,433.755,0.83,55.767398,48.156425,50.393701,58.968379,52.767447,76.763429,73.680171,66.144222,...,99.987800,0.000299,-0.000352,48.602740,44.857073,49.457995,0.059491,-0.164065,0.910007,0.080
2016-01-01 09:01:30,433.820,0.70,57.292660,49.944134,62.992126,58.888194,52.503355,76.199422,73.610442,66.124865,...,100.005879,0.000281,-0.000322,48.681475,45.071794,51.278600,0.114191,-0.065291,1.020682,0.065
2016-01-01 09:02:00,433.820,0.70,58.531935,51.396648,73.228346,58.811705,52.247333,75.787164,73.540045,66.103348,...,100.020347,0.000263,-0.000292,48.759698,45.245027,51.797603,0.158676,0.016631,1.117046,0.000
2016-01-01 09:02:30,433.840,0.68,58.531935,51.396648,73.228346,58.734962,51.991311,75.353790,73.468980,66.079675,...,100.019943,0.000245,-0.000263,48.754098,45.245027,51.409396,0.159400,0.023839,1.097442,0.020
2016-01-01 09:03:00,433.810,0.66,58.913251,51.843575,76.377953,58.659532,51.737772,75.037013,73.397243,66.053855,...,100.023786,0.000227,-0.000230,48.781820,45.298110,52.724796,0.173551,0.054326,1.121409,-0.030
2016-01-01 09:03:30,433.470,1.42,58.341277,51.173184,71.653543,58.586079,51.481047,74.581666,73.324852,66.025799,...,100.016485,0.000209,-0.000201,48.824494,45.243282,51.377410,0.153931,0.026644,1.055581,-0.340


In [8]:
data1m

,mid price,B-ASPREAD,K360,K180,K60,D360,D180,D60,sD360,sD180,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-01-01 17:58:00,433.890,0.22,64.327062,44.792833,6.451613,50.498032,69.982094,46.163657,35.312059,55.487711,...,99.823031,0.001397,0.000716,49.696396,50.951675,49.002904,-0.383073,0.939306,-1.208706,1.670
2016-01-01 17:59:00,433.630,0.28,88.494935,82.194849,96.236559,50.628973,70.186470,47.142260,35.332907,55.633581,...,100.204328,0.001435,0.000715,50.409679,52.037307,52.074236,0.203981,1.898439,1.175688,-0.260
2016-01-01 18:00:00,432.145,0.21,84.732272,76.371781,82.258065,50.743360,70.347422,47.866137,35.354415,55.781067,...,100.141045,0.001471,0.000708,50.250170,51.815642,51.516245,0.116863,1.733000,0.783942,-1.485
2016-01-01 18:01:00,433.675,0.65,63.241679,42.272727,2.419355,50.789295,70.346857,47.286765,35.376364,55.929617,...,99.800677,0.001503,0.000657,49.555518,51.104861,48.851166,-0.400494,0.890837,-1.289925,1.530
2016-01-01 18:02:00,433.675,0.65,85.383502,76.318875,84.677419,50.888511,70.529180,48.114155,35.398882,56.080393,...,100.150473,0.001544,0.000649,50.130581,51.913524,51.501796,0.142640,1.873634,0.840205,0.000
2016-01-01 18:03:00,433.465,0.57,85.383502,74.686717,84.677419,50.979768,70.689436,48.539308,35.421925,56.233382,...,100.150126,0.001585,0.000611,50.068485,51.840127,50.151007,0.147217,2.023096,0.866134,-0.210
2016-01-01 18:04:00,433.545,0.89,82.344428,69.423559,73.387097,51.060726,70.706374,48.941719,35.445450,56.387453,...,100.101091,0.001610,0.000592,49.966844,51.085590,50.235294,0.077353,1.873498,0.586303,0.080
2016-01-01 18:05:00,432.270,0.50,83.502171,71.428571,77.688172,51.148083,70.732491,49.371623,35.469191,56.542569,...,100.118987,0.001636,0.000570,50.024863,51.206266,50.252016,0.110409,1.901553,0.721374,-1.275
2016-01-01 18:06:00,432.190,0.32,65.050651,39.473684,9.139785,51.200901,70.582601,48.630645,35.493001,56.697361,...,99.829046,0.001653,0.000514,49.629995,50.479035,48.112294,-0.340857,1.123633,-1.095682,-0.080
2016-01-01 18:07:00,433.375,0.65,63.892909,37.468672,4.838710,51.255278,70.430687,47.808512,35.516962,56.851756,...,99.815430,0.001670,0.000455,49.634076,50.484726,47.963291,-0.365422,1.067191,-1.177617,1.185


In [9]:
data5m

,mid price,B-ASPREAD,K360,K180,K60,D360,D180,D60,sD360,sD180,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-01-04 17:50:00,433.490,0.72,90.944499,78.397213,77.372263,56.743089,85.371437,59.399938,39.243672,70.720565,...,100.319711,0.004104,0.001746,51.694725,54.633716,53.820598,1.898546,2.361261,1.188853,-0.340
2016-01-04 17:55:00,434.060,0.08,87.633885,70.499419,69.099757,56.926382,85.238712,58.979721,39.273027,70.971626,...,100.237052,0.004114,0.001757,51.356694,52.915489,53.114754,1.762655,2.224294,1.005724,0.570
2016-01-04 18:00:00,432.875,0.55,93.184031,83.739837,82.968370,57.125644,85.148378,59.181642,39.303316,71.219792,...,100.360694,0.004119,0.001810,51.609154,52.848875,56.261128,2.011598,2.327185,1.213850,-1.185
2016-01-04 18:05:00,433.020,0.34,81.645570,56.213705,54.136253,57.293957,84.905876,59.376962,39.334459,71.464207,...,100.080824,0.004117,0.001855,51.077195,51.298377,54.373387,1.515463,1.978535,0.670609,0.145
2016-01-04 18:10:00,432.780,0.44,83.057449,59.581882,57.664234,57.476399,84.681331,59.488902,39.366519,71.704887,...,100.108600,0.004114,0.001897,51.223614,51.461988,54.281609,1.586453,1.981865,0.708629,-0.240
2016-01-04 18:15:00,432.795,0.41,80.720545,54.006969,51.824818,57.649867,84.425814,59.373186,39.399471,71.941690,...,100.048933,0.004110,0.001927,51.095600,51.156463,53.104435,1.492640,1.888289,0.579870,0.015
2016-01-04 18:20:00,433.065,0.25,80.866602,54.355401,52.189781,57.822637,84.172610,59.871767,39.433307,72.174644,...,100.045192,0.004107,0.001987,51.093439,51.180908,55.592105,1.508988,1.861848,0.561801,0.270
2016-01-04 18:25:00,433.010,0.02,83.495618,60.627178,58.759124,57.989470,83.954250,60.365304,39.467970,72.404108,...,100.099912,0.004106,0.002047,51.106400,51.504729,55.921638,1.637370,1.899151,0.657114,-0.055
2016-01-04 18:30:00,433.590,0.14,82.960078,59.349593,57.420925,58.166952,83.728414,60.824689,39.503439,72.629886,...,100.079777,0.004102,0.002108,51.180998,51.279842,55.700326,1.623114,1.855127,0.606907,0.580
2016-01-04 18:35:00,433.760,0.56,88.607595,72.822300,71.532847,58.343847,83.584112,61.574565,39.539664,72.852672,...,100.203891,0.004105,0.002185,51.303772,52.079280,57.401552,1.888167,1.966404,0.844467,0.170


In [10]:
data10m

,mid price,B-ASPREAD,K360,K180,K60,D360,D180,D60,sD360,sD180,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-01-08 11:40:00,454.460,0.54,76.750204,63.195343,36.960179,71.388015,87.945768,28.808812,67.167271,79.685439,...,99.810040,0.024502,0.004456,55.762445,55.634688,40.847185,1.266263,1.569352,-1.920516,-0.385
2016-01-08 11:50:00,454.430,0.48,75.701444,61.376921,32.641615,71.401374,87.810912,29.139863,67.193589,79.867975,...,99.723113,0.024535,0.004306,55.606402,55.424618,50.895243,1.242570,1.544214,-1.994630,-0.030
2016-01-08 12:00:00,454.740,0.12,75.619722,58.245860,32.305104,71.409551,87.655318,28.858212,67.219561,80.048464,...,99.728803,0.024568,0.004011,55.555834,55.379759,44.969353,1.234163,1.694912,-1.907257,0.310
2016-01-08 12:10:00,455.250,0.46,76.464179,58.935361,35.782389,71.419552,87.445131,28.528406,67.245185,80.226287,...,99.810544,0.024583,0.003719,55.620252,54.916804,44.320073,1.240270,1.759754,-1.714302,0.510
2016-01-08 12:20:00,455.345,0.23,77.853446,51.024096,41.503085,71.451018,87.177090,28.293946,67.270556,80.400674,...,99.934347,0.024592,0.003443,55.921267,54.956615,45.166096,1.254839,2.218854,-1.468161,0.095
2016-01-08 12:30:00,454.800,0.40,78.112231,51.596386,42.568704,71.494157,86.908181,28.048445,67.295765,80.571364,...,99.967416,0.024531,0.003233,56.059923,52.908120,45.010457,1.251697,2.195991,-1.383188,-0.545
2016-01-08 12:40:00,455.425,0.83,76.627622,48.313253,43.449198,71.537867,86.638531,27.547799,67.320833,80.738352,...,99.868467,0.024473,0.002936,55.957590,52.963395,41.212967,1.221447,2.116711,-2.032108,0.625
2016-01-08 12:50:00,455.910,0.14,78.330155,52.078313,51.804813,71.585785,86.372300,27.633586,67.345788,80.901570,...,100.013359,0.024408,0.002774,56.090447,52.845921,46.506185,1.241040,2.145461,-1.724794,0.485
2016-01-08 13:00:00,455.375,0.25,79.651321,55.000000,58.288770,71.641806,86.122300,27.666760,67.370644,81.061015,...,100.129634,0.024344,0.002585,56.243933,53.021912,45.457644,1.254555,2.160456,-1.495674,-0.535
2016-01-08 13:10:00,455.535,0.13,78.193953,51.777108,51.794177,71.665741,85.857338,27.522721,67.395239,81.217214,...,100.025517,0.024285,0.002364,55.858936,52.843772,43.787183,1.224720,2.082784,-1.657550,0.160


# Feature Engineering of Trades Feature Set

In [6]:
def generate_trade_features(frame, freq):
    close = frame['Price']
    frame['K360'] = stoch_K(close, 360)
    frame['K180'] = stoch_K(close, 180)
    frame['K60'] = stoch_K(close, 60)
    frame['D360'] = stoch_D(frame['K360'], 360)
    frame['D180'] = stoch_D(frame['K180'], 180)
    frame['D60'] = stoch_D(frame['K60'], 60)
    frame['sD360'] = slow_D(frame['D360'], 360)
    frame['sD180'] = slow_D(frame['D180'], 180)
    frame['sD60'] = slow_D(frame['D60'], 60)
    frame['MOM360'] = momentum(close, 360)
    frame['MOM180'] = momentum(close, 180)
    frame['MOM60'] = momentum(close, 60)
    frame['ROC360'] = roc(close, 360)
    frame['ROC180'] = roc(close, 180)
    frame['ROC60'] = roc(close, 60)
    frame['LWR360'] = lw_R(close, 360)
    frame['LWR180'] = lw_R(close, 180)
    frame['LWR60'] = lw_R(close, 60)
    frame['ADOSC360'] = ad_osc(close, 360)
    frame['ADOSC180'] = ad_osc(close, 180)
    frame['ADOSC60'] = ad_osc(close, 60)
    frame['DISP360'] = disp(close, 360)
    frame['DISP180'] = disp(close, 180)
    frame['DISP60'] = disp(close, 60)
    frame['OSCP180-360'] = oscp(close, 180, 360)
    frame['OSCP60-180'] = oscp(close, 60, 180)
    frame['RSI360'] = rsi(close, 360)
    frame['RSI180'] = rsi(close, 180)
    frame['RSI60'] = rsi(close, 60)
    frame['CCI360'] = cci(close, 360)
    frame['CCI180'] = cci(close, 180)
    frame['CCI60'] = cci(close, 60)
    frame['DELTAP'] = close.diff()
    
    frame['Price'] = frame['Price'].shift(-1)
    frame['DELTAP'] = frame['DELTAP'].shift(-1)
    frame.set_index(frame.index.shift(1, freq=freq), inplace = True)
    frame = frame[3*359:-1]
    
    return frame

In [7]:
trades = pd.read_csv('../btc-data/BTC_Trades_clean.csv', parse_dates=True)
trades.drop_duplicates(subset = 'Date', inplace = True)
trades.set_index('Date', inplace = True)
trades.drop(['Trade ID', 'Amount'], axis = 1, inplace= True)
trades.index = pd.to_datetime(trades.index)
trades.sort_index(inplace= True)

trades30 = trades.reindex(pd.date_range(start = trades.index[0],
                                        end = trades.index[-1], freq='30s'),
                          method = 'nearest')

trades60 = trades.reindex(pd.date_range(start = trades.index[0],
                                        end = trades.index[-1], freq='60s'),
                          method = 'nearest')

trades300 = trades.reindex(pd.date_range(start = trades.index[0],
                                        end = trades.index[-1], freq='300s'),
                          method = 'nearest')

trades600 = trades.reindex(pd.date_range(start = trades.index[0],
                                        end = trades.index[-1], freq='600s'),
                          method = 'nearest')

In [8]:
trades_techind30 = generate_trade_features(trades30, '30s')
trades_techind30.replace([np.inf, -np.inf], np.nan, inplace = True)
trades_techind30.fillna(method='ffill', inplace = True)

/home/dl3913/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/dl3913/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2756: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [9]:
trades_techind60 = generate_trade_features(trades60, '60s')
trades_techind60.replace([np.inf, -np.inf], np.nan, inplace = True)
trades_techind60.fillna(method='ffill', inplace = True)

/home/dl3913/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/dl3913/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2756: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [10]:
trades_techind300 = generate_trade_features(trades300, '300s')
trades_techind300.replace([np.inf, -np.inf], np.nan, inplace = True)
trades_techind300.fillna(method='ffill', inplace = True)

/home/dl3913/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/dl3913/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2756: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [11]:
trades_techind600 = generate_trade_features(trades600, '60s')
trades_techind600.replace([np.inf, -np.inf], np.nan, inplace = True)
trades_techind600.fillna(method='ffill', inplace = True)

/home/dl3913/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/dl3913/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2756: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [12]:
trades_techind30.to_csv(path_or_buf='../btc-data/BTC_Trades_techind_30s.csv')
trades_techind60.to_csv(path_or_buf='../btc-data/BTC_Trades_techind_60s.csv')
trades_techind300.to_csv(path_or_buf='../btc-data/BTC_Trades_techind_300s.csv')
trades_techind600.to_csv(path_or_buf='../btc-data/BTC_Trades_techind_600s.csv')

In [13]:
path = '../btc-data/BTC_Trades_techind_30s.csv'
data30s = pd.read_csv(path, index_col = 0, parse_dates = True)

path = '../btc-data/BTC_Trades_techind_60s.csv'
data1m = pd.read_csv(path, index_col = 0, parse_dates = True)

path = '../btc-data/BTC_Trades_techind_300s.csv'
data5m = pd.read_csv(path, index_col = 0, parse_dates = True)

path = '../btc-data/BTC_Trades_techind_600s.csv'
data10m = pd.read_csv(path, index_col = 0, parse_dates = True)

In [14]:
data30s

,Price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-02-02 06:58:38,372.52,13.588850,8.148148,12.716763,62.456425,49.315279,22.895471,76.610950,75.042273,25.494617,...,99.860471,0.000729,-0.001101,49.739935,49.123461,45.067265,0.320664,-0.670980,-1.446046,0.00
2016-02-02 06:59:08,373.09,13.588850,8.148148,12.716763,62.355947,48.804991,22.680066,76.529835,74.916225,25.355198,...,99.864397,0.000715,-0.001128,50.253017,48.469966,45.067265,0.315882,-0.680146,-1.401352,0.57
2016-02-02 06:59:38,373.09,33.449477,29.259259,45.664740,62.310638,48.411987,23.013795,76.448752,74.785223,25.229498,...,100.018587,0.000705,-0.001138,50.798845,49.556213,48.366702,0.789845,-0.173456,-0.566770,0.00
2016-02-02 07:00:08,373.09,33.449477,29.259259,45.664740,62.265329,48.018983,23.625301,76.367701,74.649268,25.126773,...,100.018810,0.000695,-0.001137,50.798845,49.556213,49.729144,0.786000,-0.182919,-0.542314,0.00
2016-02-02 07:00:38,373.09,33.449477,29.259259,45.664740,62.157553,47.625979,24.236808,76.286334,74.509600,25.039942,...,100.019034,0.000689,-0.001135,50.349650,49.556213,49.729144,0.781612,-0.192548,-0.517869,0.00
2016-02-02 07:01:08,373.09,33.449477,29.259259,45.664740,62.049777,47.232974,24.570536,76.204653,74.366179,24.959746,...,100.020419,0.000683,-0.001146,50.349650,49.556213,48.272018,0.777241,-0.202352,-0.492211,0.00
2016-02-02 07:01:38,373.25,33.449477,29.259259,45.664740,62.002617,46.839970,24.904265,76.122993,74.219003,24.899655,...,100.021805,0.000676,-0.001156,50.349650,49.556213,48.272018,0.772765,-0.212339,-0.466282,0.16
2016-02-02 07:02:08,373.25,39.024390,35.185185,54.913295,61.970942,46.479888,25.318233,76.041628,74.068257,24.859776,...,100.065370,0.000671,-0.001162,50.503486,49.852941,49.178532,0.904226,-0.071653,-0.189571,0.00
2016-02-02 07:02:38,373.25,39.024390,35.185185,54.913295,61.872790,46.119806,25.732202,75.960188,73.914698,24.836915,...,100.066040,0.000668,-0.001167,50.233645,49.852941,49.178532,0.899335,-0.081622,-0.162706,0.00
2016-02-02 07:03:08,373.25,39.024390,35.185185,54.913295,61.774638,45.759723,26.501224,75.878674,73.758327,24.842907,...,100.065370,0.000665,-0.001159,50.233645,49.852941,50.849377,0.894467,-0.091779,-0.145698,0.00


In [15]:
data1m

,Price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-02-02 15:57:38,375.00,96.192385,92.720307,100.000000,57.648247,78.584338,62.241765,45.923501,69.741715,46.297091,...,100.220558,0.002065,0.000282,51.665885,53.562200,57.821553,1.361923,1.657565,2.283489,0.20
2016-02-02 15:58:38,374.80,100.000000,100.000000,100.000000,57.841821,78.677735,63.613193,45.925769,69.928117,46.568136,...,100.266531,0.002082,0.000325,51.567398,53.902528,59.655172,1.569055,1.766314,2.948180,-0.20
2016-02-02 15:59:38,374.80,96.000000,92.366412,90.697674,58.069773,78.693298,64.867678,45.929089,70.112271,46.877834,...,100.206268,0.002091,0.000367,51.786558,53.135314,58.577878,1.371810,1.628471,2.347402,0.00
2016-02-02 16:00:38,374.80,96.000000,92.366412,89.473684,58.300628,78.712080,66.358906,45.933478,70.294180,47.234310,...,100.197561,0.002099,0.000428,51.811024,53.174312,61.565836,1.370528,1.612407,2.740133,0.00
2016-02-02 16:01:38,374.95,96.000000,92.366412,89.473684,58.567295,78.713150,67.152054,45.939244,70.473251,47.614295,...,100.192428,0.002103,0.000455,52.114684,52.984525,57.536042,1.366833,1.596165,2.539967,0.15
2016-02-02 16:02:38,374.49,99.000000,98.091603,97.368421,58.842295,78.736364,68.111685,45.946411,70.649552,48.013908,...,100.226542,0.002105,0.000490,52.585521,53.139919,58.656331,1.504576,1.669577,2.621804,-0.46
2016-02-02 16:03:38,374.95,89.800000,76.279070,73.157895,59.015982,78.776884,68.657224,45.954551,70.824791,48.435967,...,100.099657,0.002122,0.000497,51.514180,53.932165,55.365854,1.054976,1.665384,1.684395,0.46
2016-02-02 16:04:38,375.00,99.000000,97.674419,97.368421,59.190814,78.773626,69.575931,45.963404,70.998092,48.885770,...,100.216764,0.002130,0.000533,51.645893,52.968037,57.589803,1.500395,1.922833,2.366635,0.05
2016-02-02 16:05:38,375.00,100.000000,100.000000,100.000000,59.383575,78.802611,70.672188,45.973036,71.169574,49.368225,...,100.223387,0.002138,0.000574,51.833467,53.300267,58.851114,1.544356,1.929849,2.307969,0.00
2016-02-02 16:06:38,374.49,100.000000,100.000000,100.000000,59.576336,78.823545,72.116038,45.983437,71.339146,49.894920,...,100.214905,0.002146,0.000634,51.833467,53.211009,61.670762,1.539678,1.906751,2.659205,-0.51


In [16]:
data5m

,Price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-02-05 15:49:38,387.99,83.853119,82.795699,71.830986,85.563941,60.423892,76.702545,68.261256,74.036084,80.094739,...,100.035825,0.006598,0.002359,55.570264,50.512645,52.364360,0.953797,0.601820,0.904864,-0.61
2016-02-05 15:54:38,388.16,80.784708,69.677419,50.352113,85.557149,60.361905,76.234055,68.367920,73.869704,80.284157,...,99.875838,0.006488,0.002388,55.328893,49.991544,51.695332,0.900988,0.473635,0.399187,0.17
2016-02-05 15:59:38,388.50,81.639839,73.333333,56.338028,85.553423,60.325818,75.706355,68.474420,73.703736,80.452210,...,99.917242,0.006381,0.002407,55.389201,50.304569,51.360713,0.911648,0.498754,0.518882,0.34
2016-02-05 16:04:38,388.64,83.350101,80.645161,57.547170,85.558243,60.324764,75.737270,68.580787,73.538294,80.620222,...,99.996439,0.006274,0.002483,55.536332,50.422012,54.974359,0.936500,0.557901,1.543157,0.14
2016-02-05 16:09:38,388.39,84.054326,83.655914,61.616162,85.565709,60.312769,75.466770,68.687032,73.373154,80.774541,...,100.026638,0.006153,0.002546,55.587056,49.700291,53.594080,0.945092,0.577522,1.902187,-0.25
2016-02-05 16:14:38,388.39,82.796781,78.279570,48.989899,85.563471,60.266715,74.852499,68.793109,73.208173,80.912651,...,99.958650,0.006029,0.002593,55.435759,49.358865,52.247488,0.921297,0.518968,1.723290,0.00
2016-02-05 16:19:38,388.54,82.796781,78.279570,48.989899,85.560888,60.236030,74.002330,68.899017,73.043521,81.026849,...,99.959979,0.005914,0.002582,55.432679,49.827408,49.126761,0.918391,0.510564,1.680609,0.15
2016-02-05 16:24:38,388.80,83.551308,81.505376,56.565657,85.560401,60.221310,73.278425,69.004710,72.879276,81.117888,...,100.000043,0.005798,0.002570,55.478837,49.896587,49.040632,0.928249,0.532641,1.800482,0.26
2016-02-05 16:29:38,388.96,84.859155,87.096774,69.696970,85.565272,60.246596,72.773374,69.110205,72.715688,81.194719,...,100.067475,0.005689,0.002558,55.577772,50.396825,49.665552,0.947693,0.577321,2.031600,0.16
2016-02-05 16:34:38,388.98,85.663984,90.537634,77.777778,85.573068,60.300220,72.410997,69.215510,72.552955,81.258969,...,100.108398,0.005585,0.002547,55.634385,50.821942,50.165929,0.958871,0.601752,2.128316,0.02


In [17]:
data10m

,Price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60,CCI360,CCI180,CCI60,DELTAP
2016-02-09 09:30:38,371.89,24.373119,27.898967,81.543624,24.939969,33.397432,36.749782,39.588132,42.888992,21.088897,...,100.222627,-0.001985,-0.006983,49.257669,47.167102,51.938308,-1.500696,-0.951494,-0.529665,-0.56
2016-02-09 09:40:38,371.96,18.756269,21.469575,62.751678,24.931137,33.240129,37.646787,39.471033,42.849181,21.469511,...,100.067494,-0.002025,-0.006869,48.807328,47.001431,52.068533,-1.575963,-1.064863,-0.797692,0.07
2016-02-09 09:50:38,372.59,19.458375,22.273249,65.100671,24.917242,33.125660,38.556359,39.354255,42.808238,21.866422,...,100.082020,-0.002053,-0.006765,48.677966,47.345017,52.008368,-1.554966,-1.037644,-0.735523,0.63
2016-02-09 10:00:38,373.64,25.777332,29.506315,86.241611,24.944278,33.014604,39.680057,39.237963,42.765692,22.282275,...,100.246722,-0.002096,-0.006659,49.361359,47.461830,52.228238,-1.449488,-0.886982,-0.382604,1.05
2016-02-09 10:10:38,373.02,36.308927,41.561424,100.000000,24.998523,32.970520,41.022428,39.122244,42.721940,22.725655,...,100.519853,-0.002129,-0.006523,49.645478,48.171988,54.156675,-1.281893,-0.644424,0.530207,-0.62
2016-02-09 10:20:38,373.79,30.090271,34.443169,82.872928,25.018837,32.886891,42.239523,39.006958,42.676966,23.197143,...,100.343607,-0.002159,-0.006376,49.100019,47.790549,54.200000,-1.367875,-0.774147,0.205859,0.77
2016-02-09 10:30:38,373.00,37.813440,43.283582,100.000000,25.084567,32.890745,43.586439,38.892290,42.631209,23.687051,...,100.540282,-0.002188,-0.006238,49.844527,48.593969,54.599524,-1.243644,-0.594310,0.706621,-0.79
2016-02-09 10:40:38,373.27,29.889669,34.213548,79.045093,25.117766,32.828222,44.552980,38.778124,42.584371,24.187219,...,100.321405,-0.002221,-0.006127,49.375272,47.979288,52.741313,-1.355192,-0.762556,0.294993,0.27
2016-02-09 10:50:38,373.98,32.597793,37.313433,86.206897,25.143292,32.794112,45.579462,38.664409,42.536877,24.703312,...,100.387364,-0.002242,-0.006018,49.131793,48.241079,52.850539,-1.306544,-0.693782,0.457675,0.71
2016-02-09 11:00:38,373.42,39.719157,45.464983,100.000000,25.203795,32.810085,46.841488,38.551285,42.488827,25.235179,...,100.568349,-0.002264,-0.005888,49.677662,48.736868,54.146341,-1.192288,-0.527301,0.958244,-0.56
